In [ ]:
pip install scikit-image
from sort import Sort

In [ ]:
# Célula 1: Configuração do ambiente e importações

# Importações necessárias para todo o notebook
import os
import sys  # Necessário para modificar o sys.path
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO  # Framework para modelos YOLO
from IPython.display import display, Image

# Adiciona o caminho da pasta 'sort' ao sys.path para que possamos importar o módulo
sort_path = r'C:\Users\wasse\Documentos\Modelo-YOLO-V8\sort'  # Caminho para a pasta 'sort'
sys.path.append(sort_path)  # Adiciona ao sys.path

# Verifica se a pasta foi adicionada corretamente
print("Conteúdo da pasta 'sort':", os.listdir(sort_path))

# Agora podemos importar o módulo 'Sort' corretamente
try:
    from sort import Sort  # Algoritmo de tracking SORT (importado do repositório original)
    print("Módulo 'Sort' importado com sucesso!")
except ImportError as e:
    print(f"Erro ao importar o módulo 'Sort': {e}")


In [ ]:
# Célula 2: Definição dos caminhos dos dados e configuração
# Caminho base do dataset (alterado conforme meu diretório local)
dataset_path = r'C:\Users\wasse\Documentos\Modelo-YOLO-V8'
train_path = os.path.join(dataset_path, 'train', 'images')
val_path = os.path.join(dataset_path, 'valid', 'images')
test_path = os.path.join(dataset_path, 'test', 'images')

# Exibindo algumas informações sobre os dados
print("Caminho do dataset:", dataset_path)
print("Imagens de treino:", len(os.listdir(train_path)))
print("Imagens de validação:", len(os.listdir(val_path)))
print("Imagens de teste:", len(os.listdir(test_path)))


In [ ]:
# Célula 3: Configuração e treinamento do modelo YOLOv8
# Carrega um modelo YOLOv8 pré-treinado para utilizar como base
model = YOLO('yolov8n.pt')  # Usando a versão "nano" para treinar mais rápido

# Treinamento do modelo no meu dataset customizado por 15 épocas
model.train(
    data=os.path.join(dataset_path, 'data.yaml'),  # Arquivo YAML com as classes e caminhos
    epochs=15,  # Definindo 15 épocas para acelerar o processo
    imgsz=640,  # Tamanho das imagens para treinamento
    batch=8,  # Tamanho do lote, ajustado para evitar problemas de memória
    name='YOLOv8_Better_Beef_Training',  # Nome do projeto de treinamento
    save=True  # Salvando checkpoints durante o treinamento
)


In [ ]:
# Célula 4: Carregamento do modelo treinado para inferência
model_path = r'C:\Users\wasse\Documentos\Modelo-YOLO-V8\yolov8_env\Include\runs\detect\YOLOv8_Better_Beef_Training\weights\best.pt'

# Caminho do vídeo de entrada (alterado para meu diretório local)
video_path = r'C:\Users\wasse\Documentos\Modelo-YOLO-V8\imagem_termica1.mp4'

# Definindo intervalo para cortar do meio do vídeo (1.5 minutos, do meio)
start_time = 1200  # Início do corte em segundos (20 minutos)
end_time = start_time + 90  # Final do corte (21.5 minutos)
output_video_path = 'output_segment.mp4'  # Nome do arquivo de saída com o corte

# Função para cortar o vídeo no intervalo desejado
def cut_video(input_path, output_path, start, end):
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Obtém o FPS do vídeo original
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    current_frame = 0  # Inicializa contagem de frames
    start_frame = int(start * fps)  # Frame inicial do corte
    end_frame = int(end * fps)  # Frame final do corte
    
    # Loop para percorrer o vídeo e realizar o corte
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or current_frame > end_frame:  # Interrompe se atingir o final do corte
            break
        if current_frame >= start_frame:  # Escreve os frames dentro do intervalo
            out.write(frame)
        current_frame += 1
    
    # Libera os recursos de vídeo
    cap.release()
    out.release()
    print(f"Segmento de vídeo salvo em: {output_path}")  # Informa a conclusão do corte

# Chamando a função para cortar o vídeo
cut_video(video_path, output_video_path, start_time, end_time)


In [ ]:
# Célula 5: Tracking dos animais no vídeo cortado
# Caminho de entrada e saída para o vídeo com tracking
input_video_path = output_video_path  # Vídeo de entrada é o segmento cortado
output_tracked_video = 'tracked_video.mp4'  # Nome do vídeo de saída com tracking

# Inicializa a captura do vídeo e parâmetros de escrita
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)  # FPS do vídeo cortado
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_tracked_video, fourcc, fps, (width, height))

# Inicializa o algoritmo de tracking SORT para manter o ID dos animais
tracker = Sort()

# Variável para contagem de animais (IDs únicos)
animal_count = 0

# Loop para processar cada frame do vídeo
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:  # Interrompe quando acaba o vídeo
        break
    
    # Realiza a inferência no frame usando o modelo treinado
    results = model(frame)
    
    # Extrai bounding boxes e scores das detecções
    detections = []
    for result in results:
        for box in result.boxes:
            if len(box.xyxy) > 0:  # Verifica se há detecções
                x1, y1, x2, y2 = box.xyxy.cpu().numpy().astype(int)[0]  # Coordenadas da bounding box
                conf = box.conf.cpu().numpy()[0]  # Confiança da detecção
                detections.append([x1, y1, x2, y2, conf])  # Adiciona ao array de detecções
    
    # Converte para numpy array e verifica se há detecções
    detections = np.array(detections)
    if len(detections) > 0:  # Se houver detecções, faz o tracking
        tracked_objects = tracker.update(detections)
        
        # Desenha bounding boxes e IDs nos frames
        for obj in tracked_objects:
            x1, y1, x2, y2, obj_id = obj.astype(int)  # Convertendo para int
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Desenha a caixa
            cv2.putText(frame, f"ID {int(obj_id)}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        
        # Atualiza a contagem de animais com base nos IDs únicos
        animal_count = len(set(tracked_objects[:, 4]))
    
    # Escreve o frame com tracking no vídeo de saída
    out.write(frame)
    
    # Mostra o frame processado (útil para visualizar durante a execução)
    cv2.imshow('Frame com Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Para visualização rápida, usa 'q' para sair
        break

# Libera os recursos de vídeo
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Vídeo processado e salvo em: {output_tracked_video}")
print(f"Total de animais rastreados: {animal_count}")  # Exibe contagem final de animais


In [ ]:
# Célula 6: Exibição e análise final do tracking e contagem
# Função para exibir o vídeo processado no notebook
def show_video(video_path, width=640, height=480):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:  # Interrompe quando acaba o vídeo
            break
        frame = cv2.resize(frame, (width, height))  # Redimensiona para visualização
        display(Image(data=cv2.imencode('.jpg', frame)[1].tobytes()))  # Mostra frame
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Exibir o vídeo processado com tracking
show_video(output_tracked_video)

# Contagem total de animais rastreados no vídeo
print(f"Contagem total de animais rastreados: {animal_count}")


In [ ]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
from sort import Sort

# Configuração do modelo e parâmetros
model_path = r'C:\Users\wasse\Documentos\Modelo-YOLO-V8\yolov8_env\Include\runs\detect\YOLOv8_Better_Beef_Training\weights\best.pt'
model = YOLO(model_path)  # Carrega o modelo treinado

# Caminhos de vídeo
input_video_path = r'C:\Users\wasse\Documentos\Modelo-YOLO-V8\imagem_termica1.mp4'
output_video_path = 'tracked_video_segment.mp4'

# Inicializa a captura do vídeo
cap = cv2.VideoCapture(input_video_path)
fps = 10  # Define um novo FPS menor para processamento (10 quadros por segundo)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Inicializa o tracker SORT
tracker = Sort()

# Parâmetros de detecção
conf_threshold = 0.5  # Confiança mínima para considerar uma detecção válida
frame_count = 0  # Contador de frames processados

# Contadores para cabeças e olhos
total_cabecas_detectadas = 0
total_olhos_detectados = 0

# Classes de detecção (supondo que as classes estão definidas no modelo YOLO)
classe_cabeca = 0  # Supondo que a classe '0' representa cabeça
classe_olho = 1  # Supondo que a classe '1' representa olho

# Posição inicial e final do segmento a ser processado (em segundos)
start_time = 4 * 60  # Minuto 4
end_time = 6 * 60  # Minuto 6

# Definir o ponto inicial do vídeo em frames
cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)  # Define o ponto inicial em milissegundos

# Loop para processar cada frame dentro do segmento desejado
while cap.isOpened():
    current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Tempo atual em segundos
    if current_time > end_time:  # Interrompe se atingir o final do segmento
        break

    ret, frame = cap.read()
    if not ret:  # Interrompe se não houver mais frames
        break
    
    # Processa apenas a cada 3 frames para reduzir a carga
    if frame_count % 3 == 0:  # Modifique para alterar a frequência de processamento
        # Realiza a inferência no frame
        results = model(frame)
        
        # Coleta as detecções
        detections = []
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy.cpu().numpy().astype(int)[0]
                conf = box.conf.cpu().numpy()[0]
                cls = int(box.cls.cpu().numpy()[0])  # Classe da detecção
                
                # Filtra as detecções com base na confiança mínima
                if conf >= conf_threshold:
                    detections.append([x1, y1, x2, y2, conf])
                    
                    # Incrementa o contador de acordo com a classe detectada
                    if cls == classe_cabeca:
                        total_cabecas_detectadas += 1
                    elif cls == classe_olho:
                        total_olhos_detectados += 1
        
        # Converte para numpy array e atualiza o tracker
        detections = np.array(detections)
        if len(detections) > 0:
            tracked_objects = tracker.update(detections)
            
            # Desenha as caixas e IDs rastreados
            for obj in tracked_objects:
                x1, y1, x2, y2, obj_id = obj.astype(int)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"ID {int(obj_id)}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    # Escreve o frame processado no vídeo de saída
    out.write(frame)
    
    # Mostra o frame processado
    cv2.imshow('Frame com Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Pressione 'q' para sair
        break
    
    frame_count += 1  # Incrementa o contador de frames

# Libera os recursos de vídeo
cap.release()
out.release()
cv2.destroyAllWindows()

# Exibe as contagens finais de detecções
print(f"Vídeo processado e salvo em: {output_video_path}")
print(f"Total de cabeças detectadas: {total_cabecas_detectadas}")
print(f"Total de olhos detectados: {total_olhos_detectados}")
